In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from utils.elasticsearch_utils import ElasticSearchUtils

/home/workspace/data-center/utils/elasticsearch_utils.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
def dump_data(index):
    host_info = {
        'host': 'https://corpus.ncsoft.com:9200',
        'http_auth': 'elastic:nlplab',
    }

    utils = ElasticSearchUtils(**host_info)

    doc_list = []
    utils.export(index=index, result=doc_list)

    return doc_list

In [42]:
# index = 'crawler-bbs-game-bns-eng'
# index = 'crawler-bbs-game-lineagem-eng'
index = 'crawler-bbs-game-lineagem-jap'

doc_list = dump_data(index=index)

print('{:,}'.format(len(doc_list)))


23,075


In [94]:
def flatten(doc_list, column):
    result = []
    for doc in doc_list:
        if column not in doc or len(doc[column]) == 0:
            continue
            
        for item in doc[column]:
            item.update(doc)
            
            result.append(item)
    
    for i in range(len(result)):
        if column in result[i]:
            del result[i][column]
        
        if 'html_content' in result[i]:
            del result[i]['html_content']
            
    return result

In [45]:
# doc2_list = flatten(doc_list=doc_list, column='articles')
doc2_list = flatten(doc_list=doc_list, column='reply_list')

len(doc2_list)

155511

In [47]:
df = pd.DataFrame(doc2_list)

df.fillna('', inplace=True)
    
print('{:,}'.format(len(df)))
df.head()

155,511


,author,date,content,title,url,views,likes,document_id,curl_date
0,グンダルバウセ,2019-12-15T20:51:00+09:00,落ちた～～落ちた～～ヴァラ鯖落ちた～～～昔は、鯖状況ってのがあったけど・・・今は、スクリティ...,ヴァラ落ちたが、さて,https://www.ncsoft.jp/lineage2/community/linea...,423,423,29142,
1,グンダルバウセ,2019-12-15T20:51:00+09:00,落ちた～～落ちた～～ヴァラ鯖落ちた～～～昔は、鯖状況ってのがあったけど・・・今は、スクリティ...,ヴァラ落ちたが、さて,https://www.ncsoft.jp/lineage2/community/linea...,423,423,29142,
2,よるん,2019-12-15T20:48:00+09:00,ヴァラサバのみ落ちてますか？切断されて点検中ってなってますけど・・\n\t\t\t\t\t\...,ヴァラサバのみ点検中？,https://www.ncsoft.jp/lineage2/community/linea...,648,648,29141,
3,よるん,2019-12-15T20:48:00+09:00,ヴァラサバのみ落ちてますか？切断されて点検中ってなってますけど・・\n\t\t\t\t\t\...,ヴァラサバのみ点検中？,https://www.ncsoft.jp/lineage2/community/linea...,648,648,29141,
4,よるん,2019-12-15T20:48:00+09:00,ヴァラサバのみ落ちてますか？切断されて点検中ってなってますけど・・\n\t\t\t\t\t\...,ヴァラサバのみ点検中？,https://www.ncsoft.jp/lineage2/community/linea...,648,648,29141,


In [48]:
df.to_excel('data/{}.xlsx'.format(index), index=False)

In [60]:
def dump_category(index):
    host_info = {
        'host': 'https://corpus.ncsoft.com:9200',
        'http_auth': 'elastic:nlplab',
    }

    utils = ElasticSearchUtils(**host_info)
    
    query = {
        '_source': ['cafe_name', 'clubid']
    }

    doc_list = []
    utils.export(index=index, result=doc_list, query=query, size=5000)

    return doc_list

In [61]:
category_list = dump_category(index='crawler-bbs-naver')

len(category_list)

3178589

In [74]:
df = pd.DataFrame(category_list)

df.fillna('', inplace=True)
    
print('{:,}'.format(len(df)))
df.head()

3,178,589


,cafe_name,clubid
0,고민상담카페,10577860
1,고민상담카페,10577860
2,고민상담카페,10577860
3,고민상담카페,10577860
4,고민상담카페,10577860


In [77]:
df['clubid'] = df['clubid'].astype('str')

In [78]:
category_df = df.groupby(by=['cafe_name', 'clubid']).size().to_frame().reset_index()

category_df

,cafe_name,clubid,0
0,,,1054
1,::맨사::맨손톱을사랑하는사람들 / No.1 네일아트정보커뮤니티 대표카페,16402471,440
2,@고고캐나다 컬리지 유학후이민 자녀무상 조기유학 워킹홀리데이,18622317,20
3,Home Barista Club(Cafe'Espresso),22853449,9643
4,No.1 네일 커뮤니티 '셀프네일 놀이터',14196843,420
...,...,...,...
69,킹스레더,17216021,19657
70,"풀잎문화센터/캘리그라피,천연비누,홈패션,배우기,리본공예 강좌",22852656,440
71,프로페셔널 바리스타 스튜디오 by GCC커피작업실,10386789,200440
72,해돌(해피돌싱&올드싱글)재혼카페/이혼상담/양육비/상간소송,10450200,440


In [79]:
category_df.to_excel('data/naver-bbs.category-count.xlsx', index=False)

In [81]:
category_df = pd.read_excel('data/naver-bbs.category-count.xlsx')

category_df

,cafe_name,clubid,0
0,★ [블소 모바일] 블레이드앤소울 레볼루션 대표 공략 카페,11992273,263629
1,♦리니지2M 대표 공략카페♦ 모바일 리니지2 m,19494347,87684
2,리공카,27434888,115319
3,리공카 모바일 대표 공략 카페,10104140,2
4,리니지2 레볼루션 공식카페,28487036,200023
5,리니지2M 대표 공략 카페 [리공카],10104140,60
6,리니지2M 대표 공략 카페 ★ [리공카],29408321,15295
7,리니지2M 대표 공략 카페 ♦ LINEAGE 2 M ♦,27434888,426105
8,리니지M 공식카페 [군주/기사/마법사/요정/리니지M 아이템 거래],12109688,306421
9,리니지M 트레이드-계정/캐릭터/아이템/다이아/거래/인벤/PC버전,13396623,152236


In [84]:
def dump_bbs(index, club_id):
    host_info = {
        'host': 'https://corpus.ncsoft.com:9200',
        'http_auth': 'elastic:nlplab',
    }

    utils = ElasticSearchUtils(**host_info)
    
    query = {
        'query': {
            'match':{
                'clubid': club_id
            }            
        }
    }

    doc_list = []
    utils.export(index=index, result=doc_list, query=query, size=1000)

    return doc_list

In [101]:
for i, row in category_df.iterrows():
    doc_list = dump_bbs(index=index, club_id=row['clubid'])
    
    doc2_list = flatten(doc_list=doc_list, column='reply_list')
    print(row['clubid'], len(doc2_list))
    
    df = pd.DataFrame(doc2_list)

    df.fillna('', inplace=True)

    df.to_json(
        'data/naver-bbs.{}.json.bz2'.format(row['clubid']),
        force_ascii=False,
        compression='bz2',
        orient='records',
        lines=True,
    )    


11992273 7630
7,630



19494347 2
2



27434888 1083885
1,083,885



10104140 3
3



28487036 0
0


...


12109688 0
0



13396623 0
0



29408321 1558
1,558



26203446 0
0



11992273 7630
7,630


In [102]:
index = 'crawler-bbs-naver'

doc_list = dump_bbs(index=index, club_id='27434888')
len(doc_list)

559464

In [103]:
doc2_list = flatten(doc_list=doc_list, column='reply_list')
len(doc2_list)

1084442

In [104]:
df = pd.DataFrame(doc2_list)

df.fillna('', inplace=True)
    
print('{:,}'.format(len(df)))
df.head()

1,084,442


,date,nick,image,reply_to,text,class,title,view_count,reply_count,url,clubid,articleid,cafe_name,curl_date,document_id,content,bbs_name,replay_count
0,2018-04-30T22:41:00+09:00,여름빛,[https://mcafethumb-phinf.pstatic.net/MjAxODA0...,,들어볼게요!,comment,오늘의 추천곡 #162_Chase Me,54,7,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,27434888,750629,리공카,2019-12-24T21:19:23.116369+09:00,27434888-750629,"오늘의 추천곡 #162ㄷㄹㅋㅊ 'Chase Me'제가 정말 좋아하는 스타일이면서,한...",.,
1,2018-04-30T22:41:00+09:00,여름빛,[https://mcafethumb-phinf.pstatic.net/MjAxODA0...,,ㄷㄹㅋㅊ분들은 다 좋으신데 인지도가 부족하더라구요ㅠ,comment,오늘의 추천곡 #162_Chase Me,54,7,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,27434888,750629,리공카,2019-12-24T21:19:23.116369+09:00,27434888-750629,"오늘의 추천곡 #162ㄷㄹㅋㅊ 'Chase Me'제가 정말 좋아하는 스타일이면서,한...",.,
2,2018-04-30T22:41:00+09:00,여름빛,[https://mcafethumb-phinf.pstatic.net/MjAxODA0...,,들어볼게용~!!,comment,오늘의 추천곡 #162_Chase Me,54,7,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,27434888,750629,리공카,2019-12-24T21:19:23.116369+09:00,27434888-750629,"오늘의 추천곡 #162ㄷㄹㅋㅊ 'Chase Me'제가 정말 좋아하는 스타일이면서,한...",.,
3,2018-04-30T22:41:00+09:00,여름빛,[https://mcafethumb-phinf.pstatic.net/MjAxODA0...,,들어볼게요~~,comment,오늘의 추천곡 #162_Chase Me,54,7,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,27434888,750629,리공카,2019-12-24T21:19:23.116369+09:00,27434888-750629,"오늘의 추천곡 #162ㄷㄹㅋㅊ 'Chase Me'제가 정말 좋아하는 스타일이면서,한...",.,
4,2018-04-30T22:41:00+09:00,여름빛,[https://mcafethumb-phinf.pstatic.net/MjAxODA0...,,ㅍㅍㅇㅇ에 수록되어 있어서 들어봤는데 진짜 좋은 곡이더라구요!,comment,오늘의 추천곡 #162_Chase Me,54,7,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,27434888,750629,리공카,2019-12-24T21:19:23.116369+09:00,27434888-750629,"오늘의 추천곡 #162ㄷㄹㅋㅊ 'Chase Me'제가 정말 좋아하는 스타일이면서,한...",.,


In [93]:
df[df['reply_list'] != '']

,title,nick,view_count,reply_count,url,date,clubid,articleid,cafe_name,curl_date,document_id,image,content,html_content,reply_list,bbs_name,replay_count
160,오늘의 추천곡 #162_Chase Me,여름빛,54,7,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2018-04-30T22:41:00+09:00,27434888,750629,리공카,2019-12-24T21:19:23.116369+09:00,27434888-750629,[https://mcafethumb-phinf.pstatic.net/MjAxODA0...,"오늘의 추천곡 #162ㄷㄹㅋㅊ 'Chase Me'제가 정말 좋아하는 스타일이면서,한...","<div class=""post_cont font_zoom1"" id=""postCont...","[{'date': '2018-04-30T22:42:00+09:00', 'nick':...",.,
161,출석수3000,콜롬비아나 아이린,60,54,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2018-04-30T22:40:00+09:00,27434888,750628,리공카,2019-12-24T21:19:28.610277+09:00,27434888-750628,[https://mcafethumb-phinf.pstatic.net/MjAxODA0...,어찌하다보니 출석수 3000이조금넘었네요 목표가있다면 5자리까지가고싶네요ㅎ,"<div class=""post_cont font_zoom1"" id=""postCont...","[{'date': '2018-04-30T22:40:00+09:00', 'nick':...",.,
162,[벨나잇]\n\n \n ...,러브배주현,13,24,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2018-04-30T22:44:00+09:00,27434888,750631,리공카,2019-12-24T21:19:17.660953+09:00,27434888-750631,[],좋은꿈꾸세여~벨나있!,"<div class=""post_cont font_zoom1"" id=""postCont...","[{'date': '2018-04-30T22:45:00+09:00', 'nick':...",t,
163,[장편팬픽]\n\n \n ...,levelupforyou,29,0,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2030-04-18T00:00:00+09:00,27434888,750627,리공카,2019-12-24T21:19:33.786842+09:00,27434888-750627,[],,,[],,
244,[벨모닝]\n\n \n ...,레사벨벳,21,7,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2018-04-30T09:36:00+09:00,27434888,750193,리공카,2019-12-24T21:25:05.083096+09:00,27434888-750193,[https://mcafethumb-phinf.pstatic.net/MjAxODA0...,이 날씨에 감기가 심하게 걸려 출근은 쉬내요 ㅋㅋ 좋은건진 안좋은건지 ㅎㅎ 그래도 ...,"<div class=""post_cont font_zoom1"" id=""postCont...","[{'date': '2018-04-30T09:42:00+09:00', 'nick':...",t,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545499,레드벨벳 아이린 무보정 사진 모음 3,가나다레드벨벳,127,1,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2016-12-30T13:22:00+09:00,27434888,214106,리니지2M 대표 공략 카페 ♦ LINEAGE 2 M ♦,2020-01-02T16:04:38.681050+09:00,27434888-214106,[https://dthumb-phinf.pstatic.net/?src=%22http...,레드벨벳 아이린 무보정 사진 모음 3\n 레드벨벳 아이린 무보정 사진 공유해요...,"<div class=""post_cont font_zoom1"" id=""postCont...","[{'date': '2017-01-15T15:08:00+09:00', 'nick':...",.,
545500,지렁이 게임인가 그거 하고 있는데,오늘도 웬디,15,3,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2016-12-30T13:21:00+09:00,27434888,214105,리니지2M 대표 공략 카페 ♦ LINEAGE 2 M ♦,2020-01-02T16:04:44.321828+09:00,27434888-214105,[],닉넴이WENDY라고 적혀있었어요ㅋㅋㅋㅋ괜히 기분 좋았어요ㅋㅋ,"<div class=""post_cont font_zoom1"" id=""postCont...","[{'date': '2016-12-30T13:22:00+09:00', 'nick':...",.,
545501,레흥돋 타임,오늘도 웬디,19,3,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2030-12-16T00:00:00+09:00,27434888,214107,리니지2M 대표 공략 카페 ♦ LINEAGE 2 M ♦,2020-01-02T16:04:32.755727+09:00,27434888-214107,[],,,[],,
545542,맛점되세용,멤버다,18,18,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,2016-12-30T12:53:00+09:00,27434888,214085,리니지2M 대표 공략 카페 ♦ LINEAGE 2 M ♦,2020-01-02T16:06:33.551086+09:00,27434888-214085,[https://mcafethumb-phinf.pstatic.net/MjAxNjEy...,,"<div class=""post_cont font_zoom1"" id=""postCont...","[{'date': '2016-12-30T12:53:00+09:00', 'nick':...",.,


In [99]:
df.to_excel('data/naver-bbs.19494347.xlsx', index=False)

ValueError: This sheet is too large! Your sheet size is: 1081171, 18 Max sheet size is: 1048576, 16384

In [100]:
df.to_json(
    'data/naver-bbs.19494347.json.bz2',
    force_ascii=False,
    compression='bz2',
    orient='records',
    lines=True,
)